---
title: "Read data, Combine tables, & aggregate numbers to understand business performance"
format:
  html:
    toc: true
execute:
    eval: false
    output: true
---

## Setup

In [ ]:
%%capture
%%bash
python ./generate_data.py
python ./run_ddl.py

In [ ]:
%%sql --show
use prod.db

In this chapter, we will go over SQL basics.


## A Spark catalog can have multiple schemas, & schemas can have multiple tables

Typically database servers can have multiple databases; each database can have multiple schemas. Each schema can have multiple tables, and each table can have multiple columns.

**Note:** We use Trino, which has `catalogs` that allow it to connect with the different underlying systems. (e.g., Postgres, Redis, Hive, etc.)

In our lab, we use Trino, and we can check the available catalogs, their schemas, the tables in a schema, & the columns in a table, as shown below.



```sql
show catalogs;
show schemas IN demo; -- Catalog -> schema
show schemas IN prod; -- schema -> namespace
show tables IN prod.db -- namespace -> Table
```

Note how, when referencing the table name, we use the full path, i.e., `database.schema.table_name`. We can skip using the full path of the table if we let Trino know which schema to use by default, as shown below.

```sql
DESCRIBE lineitem;
DESCRIBE EXTENDED lineitem;
```

## Use SELECT...FROM, LIMIT, WHERE, & ORDER BY to read the required data 

The most common use for querying is to read data in our tables. We can do this using a `SELECT ... FROM` statement, as shown below.

```sql
use prod.db;

-- use * to specify all columns
SELECT
  *
FROM
  orders
LIMIT
  4

-- use column names to only read data from those columns
SELECT
  o_orderkey,
  o_totalprice
FROM
  orders
LIMIT
  4
```

However, running a `SELECT ... FROM` statement can cause issues when the data set is extensive. If you want to look at the data, use `LIMIT n` to tell Trino only to get n number of rows. 

We can use the' WHERE' clause if we want to get the rows that match specific criteria. We can specify one or more filters within the' WHERE' clause. The `WHERE` clause with more than one filter can use combinations of `AND` and `OR` criteria to combine the filter criteria, as shown below.

```sql

-- all customer rows that have c_nationkey = 20
SELECT
  *
FROM
  customer
WHERE
  c_nationkey = 20
LIMIT
  10;


-- all customer rows that have c_nationkey = 20 and c_acctbal > 1000
SELECT
  *
FROM
  customer
WHERE
  c_nationkey = 20
  AND c_acctbal > 1000
LIMIT
  10;


-- all customer rows that have c_nationkey = 20 or c_acctbal > 1000
SELECT
  *
FROM
  customer
WHERE
  c_nationkey = 20
  OR c_acctbal > 1000
LIMIT
  10;


-- all customer rows that have (c_nationkey = 20 and c_acctbal > 1000) or rows that have c_nationkey = 11
SELECT
  *
FROM
  customer
WHERE
  (
    c_nationkey = 20
    AND c_acctbal > 1000
  )
  OR c_nationkey = 11
LIMIT
  10;
```

We can combine multiple filter clauses, as seen above. We have seen examples of equals (`=`) and greater than (`>`) conditional operators. There are 6 **conditional operators**, they are

1. `<` Less than
2. `>` Greater than
3. `<=` Less than or equal to
4. `>=` Greater than or equal to
5. `=` Equal
6. `<>` and `!=` both represent Not equal (some DBs only support one of these)

Additionally, for string types, we can make **pattern matching with `like` condition**. In a `like` condition, a `_` means any single character, and `%` means zero or more characters, for example.

```sql
USE tpch.tiny;

-- all customer rows where the name has a 381 in it
SELECT * FROM customer WHERE name LIKE '%381%';

-- all customer rows where the name ends with a 381
SELECT * FROM customer WHERE name LIKE '%381';

-- all customer rows where the name starts with a 381
SELECT * FROM customer WHERE name LIKE '381%';

-- all customer rows where the name has a combination of any character and 9 and 1
SELECT * FROM customer WHERE name LIKE '%_91%';
```

We can also filter for more than one value using `IN` and `NOT IN`.

```sql
USE tpch.tiny;

-- all customer rows which have nationkey = 10 or nationkey = 20
SELECT * FROM customer WHERE nationkey IN (10,20);

-- all customer rows which have do not have nationkey as 10 or 20
SELECT * FROM customer WHERE nationkey NOT IN (10,20);
```

We can get the number of rows in a table using `count(*)` as shown below.

```sql
USE tpch.tiny;

SELECT COUNT(*) FROM customer; -- 1500
SELECT COUNT(*) FROM lineitem; -- 60175
```

If we want to get the rows sorted by values in a specific column, we use `ORDER BY`, for example.

```sql
USE tpch.tiny;

-- Will show the first ten customer records with the lowest custkey
-- rows are ordered in ASC order by default
SELECT * FROM orders ORDER BY custkey LIMIT 10; 

-- Will show the first ten customer's records with the highest custkey
SELECT * FROM orders ORDER BY custkey DESC LIMIT 10; 
```

## Combine data from multiple tables using JOINs

We can combine data from multiple tables using joins. When we write a join query, we have a format as shown below.

```sql
SELECT
    a.*
FROM
    table_a a -- LEFT table a
    JOIN table_b b -- RIGHT table b
    ON a.id = b.id
```

The table specified first (table_a) is the left table, whereas the table established second is the right table. When we have multiple tables joined, we consider the joined dataset from the first two tables as the left table and the third table as the right table (The DB optimizes our join for performance).

```sql
SELECT
    a.*
FROM
    table_a a -- LEFT table a
    JOIN table_b b -- RIGHT table b
    ON a.id = b.id
    JOIN table_c c -- LEFT table is the joined data from table_a & table_b, right table is table_c
    ON a.c_id = c.id
```

There are five main types of joins, they are:

### 1. Inner join (default): Get rows with same join keys from both tables

```sql
USE tpch.tiny;

SELECT
    o.orderkey,
    l.orderkey
FROM
    orders o
    JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY
LIMIT
    100;

SELECT
    COUNT(o.orderkey) AS order_rows_count,
    COUNT(l.orderkey) AS lineitem_rows_count
FROM
    orders o
    JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY;
-- 2477, 2477
```
**Note:** `JOIN` defaults to INNER JOIN`.

The output will have rows from orders and lineitem that found at least one matching row from the other table with the specified join condition (same orderkey and orderdate within ship date +/- 5 days). 

We can also see that 2,477 rows from orders and lineitem tables matched.

### 2. Left outer join (aka left join): Get all rows from the left table and only matching rows from the right table.

```sql
USE tpch.tiny;

SELECT
    o.orderkey,
    l.orderkey
FROM
    orders o
    LEFT JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY
LIMIT
    100;

SELECT
    COUNT(o.orderkey) AS order_rows_count,
    COUNT(l.orderkey) AS lineitem_rows_count
FROM
    orders o
    LEFT JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY;
-- 15197, 2477
```
The output will have all the rows from orders and the rows from lineitem that were able to find at least one matching row from the orders table with the specified join condition (same orderkey and orderdate within ship date +/- 5 days). 

We can also see that the number of rows from the orders table is 15,197 & from the lineitem table is 2,477. The number of rows in orders is 15000, but the join condition produces 15197 since some orders match with multiple lineitems.

### 3. Right outer join (aka right join): Get matching rows from the left and all rows from the right table.

```sql
USE tpch.tiny;

SELECT
    o.orderkey,
    l.orderkey
FROM
    orders o
    RIGHT JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY
LIMIT
    100;

SELECT
    COUNT(o.orderkey) AS order_rows_count,
    COUNT(l.orderkey) AS lineitem_rows_count
FROM
    orders o
    RIGHT JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY;
-- 2477, 60175
```
The output will have the rows from orders that found at least one matching row from the lineitem table with the specified join condition (same orderkey and orderdate within ship date +/- 5 days) and all the rows from the lineitem table.

We can also see that the number of rows from the orders table is 15,197 & from the lineitem table is 2,477.

### 4. Full outer join: Get matched and un-matched rows from both the tables.

```sql
USE tpch.tiny;

SELECT
    o.orderkey,
    l.orderkey
FROM
    orders o
    FULL OUTER JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY
LIMIT
    100;

SELECT
    COUNT(o.orderkey) AS order_rows_count,
    COUNT(l.orderkey) AS lineitem_rows_count
FROM
    orders o
    FULL OUTER JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY;
-- 15197, 60175
```
The output will have all the rows from orders that found at least one matching row from the lineitem table with the specified join condition (same orderkey and orderdate within ship date +/- 5 days) and all the rows from the lineitem table.

We can also see that the number of rows from the orders table is 15,197 & from the lineitem table is 2,477.

### 5. Cross join: Join every row in left table with every row in the right table

```sql
USE tpch.tiny;

SELECT
    n.name AS nation_name,
    r.name AS region_name
FROM
    nation n
    CROSS JOIN region r;
```

The output will have every row of the nation joined with every row of the region. There are 25 nations and five regions, leading to 125 rows in our result from the cross-join.

<!-- ![Compare 4 joins](./images/im_27_1.png){#id .class width=30px height=20px} -->
<!-- ![Cross join](./images/im_27_2.png){#id .class width=30px height=20px} -->

There are cases where we will need to join a table with itself, called a SELF-join. Let's consider an example.

1. For every customer order, get the order placed earlier in the same week (Sunday - Saturday, not the previous seven days). Only show customer orders that have at least one such order.

```sql
USE tpch.tiny;

SELECT
    o1.custkey,
    o1.totalprice,
    o1.orderdate,
    o2.totalprice,
    o2.orderdate
FROM
    orders o1
    JOIN orders o2 ON o1.custkey = o2.custkey
    AND year(o1.orderdate) = year(o2.orderdate)
    AND week(o1.orderdate) = week(o2.orderdate)
WHERE
    o1.orderkey != o2.orderkey
LIMIT
    10;
```

## Combine data from multiple rows into one using GROUP BY

Most analytical queries require calculating metrics that involve combining data from multiple rows. `GROUP BY` allows us to perform aggregate calculations on data from a set of rows recognized by values of specified column(s). For example:

1. Create a report that shows the number of orders per orderpriority segment.

```sql
USE tpch.tiny;

SELECT
    orderpriority,
    count(*) AS num_orders
FROM
    orders
GROUP BY
    orderpriority;
```

In the above query, we group the data by `orderpriority`, and the calculation `count(*)` will be applied to the rows having a specific `orderpriority` value.

The calculations allowed are typically SUM/MIN/MAX/AVG/COUNT. However, some databases have more complex aggregate functions; check your DB documentation.

### Use HAVING to filter based on the aggregates created by GROUP BY 

## Replicate IF.ELSE logic with CASE statements

We can do conditional logic in the `SELECT ... FROM` part of our query, as shown below.

```sql
USE tpch.tiny;

SELECT
    orderkey,
    totalprice,
    CASE
        WHEN totalprice > 100000 THEN 'high'
        WHEN totalprice BETWEEN 25000
        AND 100000 THEN 'medium'
        ELSE 'low'
    END AS order_price_bucket
FROM
    orders;
```

We can see how we display different values depending on the `totalprice` column. We can also use multiple criteria as our conditional criteria (e.g., totalprice > 100000 AND orderpriority = '2-HIGH').

## Stack tables on top of each other with UNION and UNION ALL, subtract tables with EXCEPT

When we want to combine data from tables by stacking them on top of each other, we use UNION or UNION ALL. `UNION` removes duplicate rows, and `UNION ALL` does not remove duplicate rows. Let's look at an example.

```sql
USE tpch.tiny;

SELECT custkey, name FROM customer WHERE name LIKE '%_91%' -- 25 rows

-- UNION will remove duplicate rows; the below query will produce 25 rows
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
UNION
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
UNION
SELECT custkey, name FROM customer WHERE name LIKE '%_91%';

-- UNION ALL will not remove duplicate rows; the below query will produce 75 rows
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
UNION ALL
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
UNION ALL
SELECT custkey, name FROM customer WHERE name LIKE '%_91%';
```

When we want to get all the rows from the first dataset that are not in the second dataset, we can use `EXCEPT`.

```sql
USE tpch.tiny;

-- EXCEPT will get the rows in the first query result that is not in the second query result, 0 rows
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
EXCEPT
SELECT custkey, name FROM customer WHERE name LIKE '%_91%';


-- The below query will result in 23 rows; the first query has 25 rows, and the second has two rows
SELECT custkey, name FROM customer WHERE name LIKE '%_91%'
EXCEPT
SELECT custkey, name FROM customer WHERE name LIKE '%191%';
```



## Sub-query: Use query instead of a table

When we want to use the result of a query as a table in another query, we use subqueries. Let's consider an example:

1. Create a report that shows the nation, how many items it supplied (by suppliers in that nation), and how many items it purchased (by customers in that nation). 

```sql
USE tpch.tiny;

SELECT
    n.name AS nation_name,
    s.quantity AS supplied_items_quantity,
    c.quantity AS purchased_items_quantity
FROM
    nation n
    LEFT JOIN (
        SELECT
            n.nationkey,
            sum(l.quantity) AS quantity
        FROM
            lineitem l
            JOIN supplier s ON l.suppkey = s.suppkey
            JOIN nation n ON s.nationkey = n.nationkey
        GROUP BY
            n.nationkey
    ) s ON n.nationkey = s.nationkey
    LEFT JOIN (
        SELECT
            n.nationkey,
            sum(l.quantity) AS quantity
        FROM
            lineitem l
            JOIN orders o ON l.orderkey = o.orderkey
            JOIN customer c ON o.custkey = c.custkey
            JOIN nation n ON c.nationkey = n.nationkey
        GROUP BY
            n.nationkey
    ) c ON n.nationkey = c.nationkey;
```

In the above query, we can see that there are two sub-queries, one to calculate the quantity supplied by a nation and the other to calculate the quantity purchased by the customers of a nation.


## Change data types (CAST) and handle NULLS (COALESCE)

Every column in a table has a specific data type. The data types fall under one of the following categories.

1. **`Numerical`**: Data types used to store numbers.
   1. Integer: Positive and negative numbers. Different types of Integer, such as tinyint, int, and bigint, allow storage of different ranges of values. Integers cannot have decimal digits.
   2. Floating: These can have decimal digits but stores an approximate value.
   3. Decimal: These can have decimal digits and store the exact value. The decimal type allows you to specify the scale and precision. Where scale denotes the count of numbers allowed as a whole & precision denotes the count of numbers allowed after the decimal point. E.g., DECIMAL(8,3) allows eight numbers in total, with three allowed after the decimal point.
2. **`Boolean`**: Data types used to store True or False values.
3. **` String`**: Data types used to store alphanumeric characters.
   1. Varchar(n): Data type allows storage of variable character string, with a permitted max length n.
   2. Char(n): Data type allows storage of fixed character string. A column of char(n) type adds (length(string) - n) empty spaces to a string that does not have n characters.
4. **`Date & time`**: Data types used to store dates, time, & timestamps(date + time).
5. **` Objects (JSON, ARRAY)`**: Data types used to store JSON and ARRAY data.

Some databases have data types that are unique to them as well. We should check the database documents to understand the data types offered.

Functions such as `DATE_DIFF` and `ROUND` are specific to a data type. It is best practice to use the appropriate data type for your columns. We can convert data types using the `CAST` function, as shown below.

```sql
USE tpch.tiny;

SELECT
    DATE_DIFF('day', '2022-10-01', '2022-10-05'); -- will fail due to in correct data type

SELECT
    DATE_DIFF(
        'day',
        CAST('2022-10-01' AS DATE),
        CAST('2022-10-05' AS DATE)
    );
```

A `NULL` will be used for that field when a value is not present. In cases where we want to use the first non-NULL value from a list of columns, we use `COALESCE` as shown below.

Let's consider an example as shown below. We can see how when `l.orderkey` is NULL; the DB uses `999999` as the output.

```sql
USE tpch.tiny;

SELECT
    o.orderkey,
    o.orderdate,
    COALESCE(l.orderkey, 9999999) AS lineitem_orderkey,
    l.shipdate
FROM
    orders o
    LEFT JOIN lineitem l ON o.orderkey = l.orderkey
    AND o.orderdate BETWEEN l.shipdate - INTERVAL '5' DAY
    AND l.shipdate + INTERVAL '5' DAY
LIMIT
    100;
```
## Use these standard inbuilt DB functions for String, Time, and Numeric data manipulation

When processing data, more often than not, we will need to change values in columns; shown below are a few standard functions to be aware of:

1. **` String functions`**
   1. **LENGTH** is used to calculate the length of a string. E.g., `SELECT LENGTH('hi');` will output 2.
   2. **CONCAT** combines multiple string columns into one. E.g., `SELECT CONCAT(clerk, '-', orderpriority) FROM ORDERS LIMIT 5;` will concatenate clear and orderpriority columns with a dash in between them.
   3. **SPLIT** is used to split a value into an array based on a given delimiter. E.g., `SELECT SPLIT(clerk, '#') FROM ORDERS LIMIT 5;` will output a column with arrays formed by splitting clerk values on `#`.
   4. **SUBSTRING** is used to get a sub-string from a value, given the start and end character indices. E.g., `SELECT clerk, SUBSTR(clerk, 1, 5) FROM orders LIMIT 5;` will get the first five (1 - 5) characters of the clerk column. Note that the indexing starts from 1 in Trino.
   5. **TRIM** is used to remove empty spaces to the left and right of the value. E.g., `SELECT TRIM(' hi ');` will output `hi` without any spaces around it. LTRIM and RTRIM are similar but only remove spaces before and after the string, respectively.
2. **` Date and Time functions`**
   1. **Adding and subtracting dates**: Is used to add and subtract periods; the format heavily depends on the DB. E.g., In Trino, the query
      ```sql
        SELECT
        date_diff('DAY', DATE '2022-10-01', DATE '2023-11-05') diff_in_days,
        date_diff('MONTH', DATE '2022-10-01', DATE '2023-11-05') diff_in_months,
        date_diff('YEAR', DATE '2022-10-01', DATE '2023-11-05') diff_in_years;
      ```
    It will show the difference between the two dates in the specified period. We can also add/subtract an arbitrary period from a date/time column. E.g., `SELECT DATE '2022-11-05' + INTERVAL '10' DAY;` will show the output `2022-11-15`.
   2. **string <=> date/time conversions**: When we want to change the data type of a string to date/time, we can use the `DATE 'YYYY-MM-DD'` or `TIMESTAMP 'YYYY-MM-DD HH:mm:SS` functions. But when the data is in a different date/time format such as `MM/DD/YYYY`, we will need to specify the input structure; we do this using `date_parse,` E.g. `SELECT date_parse('11-05-2023', '%m-%d-%Y');`. We can convert a timestamp/date into a string with the required format using `date_format`. E.g., `SELECT DATE_FORMAT(orderdate, '%Y-%m-01') AS first_month_date FROM orders LIMIT 5;` will map every orderdate to the first of their month.
   3. **Time frame functions (YEAR/MONTH/DAY)**:  When we want to extract specific periods from a date/time column, we can use these functions. E.g., `SELECT year(date '2023-11-05');` will return 2023. Similarly, we have month, day, hour, min, etc.
3. **`Numeric`**
   1. **ROUND** is used to specify the number of digits allowed after the decimal point. E.g. `SELECT ROUND(100.102345, 2);`


## Save queries as views for more straightforward reads

When we have large/complex queries that we need to run often, we can save them as views. Views are DB objects that operate similarly to a table. The OLAP DB executes the underlying query when we query a view. 

Use views to hide query complexities and limit column access (by exposing only specific table columns) for end-users.

For example, we can create a view for the nation-level report from the above section, as shown below.

```sql
USE minio.warehouse;

CREATE VIEW nation_supplied_purchased_quantity AS
SELECT
    n.name AS nation_name,
    s.quantity AS supplied_items_quantity,
    c.quantity AS purchased_items_quantity
FROM
    tpch.tiny.nation n
    LEFT JOIN (
        SELECT
            n.nationkey,
            sum(l.quantity) AS quantity
        FROM
            tpch.tiny.lineitem l
            JOIN tpch.tiny.supplier s ON l.suppkey = s.suppkey
            JOIN tpch.tiny.nation n ON s.nationkey = n.nationkey
        GROUP BY
            n.nationkey
    ) s ON n.nationkey = s.nationkey
    LEFT JOIN (
        SELECT
            n.nationkey,
            sum(l.quantity) AS quantity
        FROM
            tpch.tiny.lineitem l
            JOIN tpch.tiny.orders o ON l.orderkey = o.orderkey
            JOIN tpch.tiny.customer c ON o.custkey = c.custkey
            JOIN tpch.tiny.nation n ON c.nationkey = n.nationkey
        GROUP BY
            n.nationkey
    ) c ON n.nationkey = c.nationkey;

SELECT
    *
FROM
    nation_supplied_purchased_quantity;
```

Now the view `nation_supplied_purchased_quantity` will run the underlying query when used. Note here we use the `minio.tpch` schema because catalog `tpch` does not allow the creation of views. The `tpch` catalog comes with Trino and only allows read operations. Read more about [connectors here](https://trino.io/docs/current/overview/concepts.html#connector).

## Exercises

1. Create a report that shows the number of returns for each region name 
2. Top 10 most selling parts
3. Sellers who sell atleast one of the top 10 selling parts
4. Number of returns per order price bucket

Assume the price bucket logic is 
```sql
 CASE
        WHEN totalprice > 100000 THEN 'high'
        WHEN totalprice BETWEEN 25000
        AND 100000 THEN 'medium'
        ELSE 'low'
    END AS order_price_bucket
```

5. Average time (in days) between receiptdate and shipdate for each nation

## Recommended reading

1. https://www.startdataengineering.com/post/improve-sql-skills-de/
2. https://www.startdataengineering.com/post/n-sql-tips-de/
3. https://www.startdataengineering.com/post/advanced-sql/